In [ ]:
#import modules
from numpy import NaN
import requests
import json
import pandas as pd
import geopandas as gpd
from pprint import pprint
import os.path
import sys
import time
import random
import numpy as np
import matplotlib.pyplot as plt
import sklearn

In [ ]:
#set study area
#load ooutput area file for London
ons_oa = gpd.read_file('../../data/boundary_london/ons_oa_geo_london_valid.geojson')
#extract oa for kensington and chelsea only
kc_oa = ons_oa[ons_oa['LAD17NM'] == 'Kensington and Chelsea']
#id column
kc_oa_id_column = "OA11CD"

In [ ]:
#extract osm data for kc by each oa
list_df = []
for this_oa_id in kc_oa[kc_oa_id_column]:
    this_section = kc_oa[kc_oa[kc_oa_id_column] == this_oa_id]
    print("Working on %s:" % (this_oa_id))
    print(this_section)
    this_section_geojson = this_section.to_json()
    #query latest  data for Kenisngton and Chelsea 
    times_to_query = '2021-10-15'
    URL = 'https://api.ohsome.org/v1/elements/geometry'
    data = {"bpolys": this_section_geojson, "time": times_to_query, "properties":'tags', "filter": "type:node"}
    response = requests.post(URL, data=data)
    response_dict = response.json()
    #print(response.json())
    oa_data = pd.json_normalize(response_dict, record_path =['features'])
    list_df.append(oa_data)
#add dataframes
kc_osm_data = pd.concat(list_df) 
print(kc_osm_data)
kc_osm_data.to_csv('../../data/data_ohsome/new_kc_osm.csv')
#get column names
print(kc_osm_data.columns)